In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [37]:
df = pd.read_csv('Real estate.csv')

In [38]:
inputData = ""
outputData = ""

def initialiseData():
    global outputData
    global inputData
    outputData = df.iloc[:,7]
    inputData = df.iloc[:,1:7]

def meanData(inputData):
    return np.mean(inputData)

def sdData(inputData):
    return np.std(inputData)

initialiseData()

In [39]:
numColumns = inputData.shape[1]
startingWeight = np.zeros(numColumns+1)

In [40]:
def normaliseData(inputData, meanInput, stdInput):
    return ((inputData-meanInput)/stdInput)

In [41]:
def loss(rowsPerFold,newWeight,testInput,trainOutputStd,trainOutputMean,testOutput):
    loss =0
    j=0
    while j< rowsPerFold:
        tempRow = testInput[j]
        tempRow = np.reshape(tempRow,(np.shape(tempRow)[0],1))
        predictedNormalised = np.dot(tempRow.transpose(),newWeight)
        predicted = predictedNormalised*trainOutputStd+trainOutputMean
        loss += (int(predicted)-int(testOutput[j]))**2
        j = j+1
    loss = loss/rowsPerFold
    loss = np.sqrt(loss)
    return loss
    
    

In [42]:
def findTerms(trainInput,trainTranspose):
    term1 = np.dot(trainTranspose,trainInput)
    term2 = np.linalg.inv(term1)
    term3 = np.dot(term2,trainTranspose)
    return term1,term2,term3

def normalSolution(k,startingWeight,inputData,outputData):
    numColumns = inputData.shape[1]
    totalRows = inputData.shape[0]
    rowsPerFold = int(totalRows/k)
    averageLoss = 0
    i=0
    while i<k:
        start = i*rowsPerFold
        end = start+rowsPerFold
        testInput = inputData[start:end].to_numpy()
        testOutput = outputData[start:end].to_numpy()
        trainInput = inputData.drop(inputData.index[start:end]).to_numpy()
        trainOutput = outputData.drop(outputData.index[start:end]).to_numpy()

        tempArr = np.ones(np.shape(trainInput)[0])
        trainInput = np.insert(trainInput,0,tempArr,axis=1)
        trainOutput = np.reshape(trainOutput,(np.shape(trainOutput)[0],1))
        trainInputMean = meanData(trainInput)
        trainInputStd = sdData(trainOutput)
        trainOutputMean = meanData(trainOutput)
        trainOutputStd = sdData(trainOutput)
        
        trainInput = normaliseData(trainInput,trainInputMean,trainInputStd)
        trainOutput = normaliseData(trainOutput,trainOutputMean,trainOutputStd)

        trainTranspose = np.transpose(trainInput)
        terms = findTerms(trainInput,trainTranspose)
        newWeight = np.dot(terms[2],trainOutput)

        temp2 = np.ones(np.shape(testInput)[0])
        testInput = np.insert(testInput,0,temp2,axis=1)
        testInput = normaliseData(testInput,trainInputMean,trainInputStd)
        
        averageLoss += loss(rowsPerFold,newWeight,testInput,trainOutputStd,trainOutputMean,testOutput)
        i = i+1

    averageLoss = averageLoss/k
    return averageLoss


In [43]:
def runNormal(k):
    averageLoss = normalSolution(k,startingWeight.copy(),inputData,outputData)
    print("The loss obtained using normal equation is for",k,"folds is",averageLoss)

In [44]:
runNormal(4)

The loss obtained using normal equation is for 4 folds is 8.842205348646141
